In [1]:
import numpy as np
import keras
import h5py
from tqdm import tqdm
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
clean_data_filename = '/content/drive/MyDrive/MLCS_4/cl/valid.h5'
poisoned_data_filename = '/content/drive/MyDrive/MLCS_4/bd/bd_valid.h5'
model_filename = '/content/drive/MyDrive/MLCS_4/bd_net.h5'


def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0, 2, 3, 1))
    return x_data, y_data

In [12]:
def load_and_evaluate_model(model_filename, cl_x_test, cl_y_test, bd_x_test, bd_y_test):
    model = keras.models.load_model(model_filename)

    cl_label_p = np.argmax(model.predict(cl_x_test), axis=1)
    clean_accuracy = np.mean(np.equal(cl_label_p, cl_y_test)) * 100
    print('Clean Classification accuracy:', clean_accuracy)

    bd_label_p = np.argmax(model.predict(bd_x_test), axis=1)
    asr = np.mean(np.equal(bd_label_p, bd_y_test)) * 100
    print('Attack Success Rate:', asr)

    return model, clean_accuracy

In [13]:
model = keras.models.load_model(model_filename)
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 55, 47, 3)]          0         []                            
                                                                                                  
 conv_1 (Conv2D)             (None, 52, 44, 20)           980       ['input[0][0]']               
                                                                                                  
 pool_1 (MaxPooling2D)       (None, 26, 22, 20)           0         ['conv_1[0][0]']              
                                                                                                  
 conv_2 (Conv2D)             (None, 24, 20, 40)           7240      ['pool_1[0][0]']              
                                                                                            

In [14]:

def prune_and_save_models(model, cl_x_test, cl_y_test, bd_x_test, bd_y_test, clean_data_acc):
    model_copy = keras.models.clone_model(model)
    model_copy.set_weights(model.get_weights())
    prune_index = []
    clean_acc = []
    asrate = []
    saved_model = np.zeros(3, dtype=bool)

    layer_output = model_copy.get_layer('pool_3').output
    intermediate_model = keras.models.Model(inputs=model_copy.input, outputs=layer_output)
    intermediate_prediction = intermediate_model.predict(cl_x_test)
    temp = np.mean(intermediate_prediction, axis=(0, 1, 2))
    seq = np.argsort(temp)
    weight_0 = model_copy.layers[5].get_weights()[0]
    bias_0 = model_copy.layers[5].get_weights()[1]

    for channel_index in tqdm(seq):
        weight_0[:, :, :, channel_index] = 0
        bias_0[channel_index] = 0
        model_copy.layers[5].set_weights([weight_0, bias_0])
        cl_label_p = np.argmax(model_copy.predict(cl_x_test), axis=1)
        clean_accuracy = np.mean(np.equal(cl_label_p, cl_y_test)) * 100
        if clean_data_acc - clean_accuracy >= 2 and not saved_model[0]:
            print("The accuracy drops at least 2%, saved the model")
            model_copy.save(f'model_X=2_channel_{channel_index}.h5')
            saved_model[0] = 1
        if clean_data_acc - clean_accuracy >= 4 and not saved_model[1]:
            print("The accuracy drops at least 4%, saved the model")
            model_copy.save(f'model_X=4_channel_{channel_index}.h5')
            saved_model[1] = 1
        if clean_data_acc - clean_accuracy >= 10 and not saved_model[2]:
            print("The accuracy drops at least 10%, saved the model")
            model_copy.save(f'model_X=10_channel_{channel_index}.h5')
            saved_model[2] = 1
        clean_acc.append(clean_accuracy)
        bd_label_p = np.argmax(model_copy.predict(bd_x_test), axis=1)
        asr = np.mean(np.equal(bd_label_p, bd_y_test)) * 100
        asrate.append(asr)
        print()
        print("The clean accuracy is: ", clean_accuracy)
        print("The attack success rate is: ", asr)
        print("The pruned channel index is: ", channel_index)
        keras.backend.clear_session()

if __name__ == '__main__':
    # Load data
    cl_x_test, cl_y_test = data_loader(clean_data_filename)
    bd_x_test, bd_y_test = data_loader(poisoned_data_filename)

    # Evaluate original model
    original_model, clean_data_acc = load_and_evaluate_model(model_filename, cl_x_test, cl_y_test, bd_x_test, bd_y_test)

    # Prune and save models based on the original model
    prune_and_save_models(original_model, cl_x_test, cl_y_test, bd_x_test, bd_y_test, clean_data_acc)


361/361 [==============================] - 8s 2ms/step
Clean Classification accuracy: 98.64899974019225
361/361 [==============================] - 1s 2ms/step
Attack Success Rate: 100.0
361/361 [==============================] - 1s 2ms/step


  0%|          | 0/60 [00:00<?, ?it/s]

361/361 [==============================] - 1s 2ms/step


  2%|▏         | 1/60 [00:03<03:48,  3.87s/it]


The clean accuracy is:  98.64899974019225
The attack success rate is:  100.0
The pruned channel index is:  0
361/361 [==============================] - 1s 2ms/step


...


The clean accuracy is:  1.5501861955486274
The attack success rate is:  0.0
The pruned channel index is:  7
361/361 [==============================] - 1s 2ms/step


 98%|█████████▊| 59/60 [03:53<00:03,  3.93s/it]


The clean accuracy is:  0.7188014202823244
The attack success rate is:  0.0
The pruned channel index is:  52
361/361 [==============================] - 1s 2ms/step


100%|██████████| 60/60 [03:57<00:00,  3.96s/it]


The clean accuracy is:  0.0779423226812159
The attack success rate is:  0.0
The pruned channel index is:  57
